# **Haystack RAG para few-shot con GPT y Ollama**

In [ ]:
## recomendado python 3.10
#%pip install haystack-ai==2.2.1 trafilatura==1.10.0 qdrant-haystack==3.8.0
#%pip install ipywidgets widgetsnbextension pandas-profiling
#%pip install ollama-haystack==0.0.7

Requiere:
- Contenedor con qdrant (bd vectorial) y reportes en un archivo con la ruta data\LosCarrera_labeled\etiquetado_1-456_v1.01\train.jsonl
- Contenedor con langfuse y otro con postgres (se usa para tracing de las consultas)
- Archivo .env con:
    - OPENAI_API_KEY - asociada a una cuenta que tenga saldo en openai
    - LANGFUSE_HOST - url y puerto (si es contenedor docker, puede ser http://localhost:3000)
    - LANGFUSE_SECRET_KEY - key generada por langfuse
    - LANGFUSE_PUBLIC_KEY - key generada por langfuse
    - HAYSTACK_CONTENT_TRACING_ENABLED = True - requerido para habilitar el tracing
- Archivo con reportes en la ruta data\LosCarrera_labeled\etiquetado_1-456_v1.01\test.jsonl

Imports

In [ ]:
import os
from dotenv import load_dotenv

from haystack import Pipeline
from haystack.dataclasses import Document
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore
from haystack_integrations.components.connectors.langfuse import LangfuseConnector

# from haystack.components.converters import TextFileToDocument
# from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import OpenAIDocumentEmbedder, OpenAITextEmbedder
from haystack.components.writers import DocumentWriter
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack_integrations.components.generators.ollama import OllamaGenerator

# from haystack.components.evaluators import DocumentRecallEvaluator

import pandas as pd
load_dotenv()

### SELECCIONAR MODELO

In [ ]:
# SELECCIONAR gpt-4o, gpt-4o-mini o llama3.1
CHAT_MODEL = "gpt-4o"

### Preprocesado dataset 
strings -> documentos

Se requiere convertir los datos del jsonl (reportes etiquetados) a un formato compatible con Haystack (clase Document).

In [ ]:
# leer dataset y convertir conjuntamente en un txt y en una lista de Documents

data_path = "data/LosCarrera_labeled/etiquetado_1-456_v1.01/"
text_file_path = data_path + "train_text.txt"
jsonl_file_path = data_path + "train.jsonl"

documents = []

with open(text_file_path, "w") as text_file:
    for i, row in pd.read_json(jsonl_file_path, lines=True).iterrows():
        text_file.write("Reporte: "+ row["text"] + "\n")
        # eliminar entidades de tipo "GANGLIOS"
        entities = [entity for entity in row["entities"] if entity["label"] != "GANGLIOS"]    
        text_file.write("Entidades: " + str(entities) + "\n")
        text_file.write("Relaciones: " + str(row["relations"]) + "\n\n\n")
        documents.append(Document(content=row["text"], meta={"id": row["id"], "entities": entities, "relations": row["relations"]}))

Ver un reporte random extraído

In [ ]:
import random

random_num = random.randint(0, len(documents)-1)

print("DOCUMENTO:\n")
print(documents[random_num])

print("\n\nCONTENIDO:\n")
print(documents[random_num].content)

### Embedding pipeline (qdrant y OpenAI)

documentos -> vectores

Se crea un pipeline que permite subir los datos a Qdrant. Actualmente, se puede montar qdrant en un docker, asegurándose que use el puerto configurado abajo.

El pipeline usa el modelo de OpenAI para generar los vectores (text-embedding-3-large), usando la librería OpenAIDocumentEmbedder.

In [ ]:
# setear y correr pipeline de indexado en la BD

document_store = QdrantDocumentStore(url="http://localhost",
                                     port=6333,
                                     embedding_dim=3072,
                                     index="AIMA_LosCarrera_V1.01")

if document_store.count_documents() == 0:
    embedder = OpenAIDocumentEmbedder(model="text-embedding-3-large")
    writer = DocumentWriter(document_store=document_store)

    indexing_pipeline = Pipeline()
    indexing_pipeline.add_component("tracer", LangfuseConnector("Qdrant Document Embedder"))
    indexing_pipeline.add_component("embedder", embedder)
    indexing_pipeline.add_component("writer", writer)

    indexing_pipeline.connect("embedder.documents", "writer.documents")
    indexing_pipeline.connect("embedder.documents", "writer.documents")

    result = indexing_pipeline.run(data={"documents":documents})

In [ ]:
document_store.count_documents()

### RAG pipeline

Embeddings - OpenAI (se usa para convertir el nuevo reporte a vector y hacer RAG)<br>
Chat - OpenAI o LLama (genera el etiquetado una vez hecho el RAG)<br>
BD vectorial - Qdrant<br>

In [ ]:
# Setear pipeline y prompt
# from haystack.components.validators import JsonSchemaValidator

NUM_EXAMPLES = 0

text_embedder = OpenAITextEmbedder(model="text-embedding-3-large")
retriever = QdrantEmbeddingRetriever(document_store ,top_k=NUM_EXAMPLES)
if CHAT_MODEL == "gpt-4o-mini" or CHAT_MODEL == "gpt-4o":
    if NUM_EXAMPLES > 0:
        template = open(data_path + "NER/prompt_gpt.txt", "r").read()
    elif NUM_EXAMPLES == 0:
        template = open(data_path + "NER/prompt_gpt_zero-shot.txt", "r").read()
        retriever = QdrantEmbeddingRetriever(document_store ,top_k=1)
    llm = OpenAIGenerator(model=CHAT_MODEL)
elif CHAT_MODEL == "llama3.1":
    template = open(data_path + "NER/prompt_ollama.txt", "r").read()
    llm = OllamaGenerator(model=CHAT_MODEL, url="http://localhost:11434/api/generate")
else:
    raise ValueError("CHAT_MODEL debe ser 'gpt-4o', 'gpt-4o-mini' o 'llama3.1'")

prompt_builder = PromptBuilder(template=template)
rag_pipeline = Pipeline()

rag_pipeline.add_component("tracer", LangfuseConnector("Mammography Few-Shot RAG NER "+ CHAT_MODEL))
rag_pipeline.add_component("text_embedder", text_embedder)
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)
# rag_pipeline.add_component("schema_validator", JsonSchemaValidator())

rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
rag_pipeline.connect("retriever.documents", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")
# rag_pipeline.connect("llm.response", "schema_validator.data")

#### Prompts a la API del modelo

Definir dataset a enviar mediante prompts

In [ ]:
reports_file_path = data_path + "test.jsonl"

# load validation data

prompt_reports = pd.read_json(reports_file_path, lines=True)

##### Loop de envío

In [ ]:
responses = []
# Setear número de reportes a enviar a la API, es un prompt por reporte
NUM_REPORTS = 69 # 69 es el total del conjunto de test

# 
processed_ids_path = "processed_ids.txt"
if os.path.exists(processed_ids_path):
    with open(processed_ids_path, "r") as f:
        processed_ids_list = f.read().splitlines()
else:
    processed_ids_list = []

for i, informe in enumerate(prompt_reports["text"][:NUM_REPORTS]):
    query = informe
    report_id = prompt_reports["id"][i]
    # si el id del reporte ya está en la lista de ids procesados, no se envía a la API
    if report_id in processed_ids_list:
        print("Reporte ", i+1, " de ", NUM_REPORTS, " ya procesado")
        continue
    result = rag_pipeline.run(data={"prompt_builder": {"query":query}, "text_embedder": {"text": query}})
    # if the string contains ```json and ``` remove them
    result_str = result["llm"]["replies"][0]
    result_str = result_str.replace("```json", "")
    result_str = result_str.replace("```", "")
    # convert result from string to a list of dictionaries
    try:
        result_json = eval(result_str)
        result_json = {"id": report_id, "entities": result_json}
        responses.append(result_json)
        print("Procesado reporte ", i+1, " de ", NUM_REPORTS)
        processed_ids_list.append(report_id)
    except:
        print("Error en el reporte ", i+1, " de ", NUM_REPORTS)
    print("id: ", report_id)
    # print(informe)
    # print(json.dumps(result_json, indent=4, ensure_ascii=False))

# Guardar el listado de ids procesados en un archivo txt en la carpeta actual
with open(processed_ids_path, "w") as f:
    for item in processed_ids_list:
        f.write("%s\n" % item)
print("Número de reportes enviados a la API: ", len(responses))

### Arreglar Spans

In [ ]:
import re
import Levenshtein

"""
    fix_spans
    Explicacion : función que recibe un conjunto de entidades (resultado de un NER) y el texto asociado, y corrige los spans de las entidades para que coincidan con las posiciones de las palabras en el texto. La manera de hacerlo es buscar el texto de la entidad en el texto asociado, y encontrar la posición de la primera y última palabra de la entidad en el texto. Si hay más de una coincidencia, se elige la que esté más cerca de la posición original de la entidad. Si no hay coincidencias, se busca la primera palabra del span de la entidad en el texto, y se busca la coincidencia más cercana a la posición original de la entidad, luego, se compara la distancia de Levenshtein entre el texto de la entidad y el texto original comenzando desde la posición encontrada (con un largo igual al largo del span de la entidad), si la distancia es menor a un umbral, se considera que se encontró la posición correcta y se ajusta el span de la entidad.

    Input : 
    - report_entities : lista de diccionarios, donde cada diccionario tiene 4 llaves: "label", "start_offset", "end_offset" y "span_text".
    - full_text : string, texto completo del que se extrajeron las entidades.

    Output :
    - fixed_entities : lista de diccionarios, donde cada diccionario tiene 4 llaves: "label", "start_offset", "end_offset" y "span_text", con los spans corregidos.
"""

def fix_spans(report_entities, full_text):
    span_difference_threshold = 5
    fixed_entities = []

    for i, entity in enumerate(report_entities):
    
        span_text = entity["span_text"]
        start_pos = entity["start_offset"]
        end_pos = entity["end_offset"]        

        # Buscar coincidencia exacta del texto de la entidad en el texto completo
        if span_text in full_text:
            # si hay más de una coincidencia, elegir la que esté más cerca de la posición original de la entidad
            # Usar escape para que no haya problemas con caracteres especiales
            escaped_span_text = re.escape(span_text)
            start_positions = [m.start() for m in re.finditer(escaped_span_text, full_text)]
            if len(start_positions) > 1:
                distances = [abs(start_pos - pos) for pos in start_positions]
                start_pos = start_positions[distances.index(min(distances))]
            else:
                start_pos = start_positions[0]
            end_pos = start_pos + len(span_text)
            fixed_entities.append({"label": entity["label"], "start_offset": start_pos, "end_offset": end_pos, "span_text": span_text})
            print("Corregido mediante coincidencia exacta")
            print("Start positions: ", start_positions)
            print("Entidad: ", span_text)
        else:
            # Extraer la primera palabra del span de la entidad
            first_word = span_text.split()[0]
            # Buscar la primera palabra en el texto completo
            if first_word in full_text:
                # si hay más de una coincidencia, elegir la que esté más cerca de la posición original de la entidad
                # Usar escape para que no haya problemas con caracteres especiales
                escaped_first_word = re.escape(first_word)
                start_positions = [m.start() for m in re.finditer(escaped_first_word, full_text)]
                if len(start_positions) > 1:
                    distances = [abs(start_pos - pos) for pos in start_positions]
                    start_pos = start_positions[distances.index(min(distances))]
                end_pos = start_pos + len(span_text)

                if end_pos > len(full_text):
                    end_pos = len(full_text)

                # Comparar la distancia de Levenshtein entre el texto de la entidad y el texto original comenzando desde la posición encontrada
                # si la distancia es menor a un umbral, se considera que se encontró la posición correcta
                if Levenshtein.distance(full_text[start_pos:end_pos], span_text) < span_difference_threshold:
                    fixed_entities.append({"label": entity["label"], "start_offset": start_pos, "end_offset": end_pos, "span_text": span_text})
                    print("Corregido mediante Levenshtein")
                else:
                    print("No se pudo corregir", span_text, start_pos, end_pos)
            else:
                print("No se pudo corregir", span_text, start_pos, end_pos)
    return fixed_entities

##### Arreglar spans y guardar respuestas en una lista

In [ ]:
import json

fixed_responses = []

for i, response_object in enumerate(responses):
    print("\n\nInforme: ", i)
    response_entities = response_object["entities"]  
    fixed_entities = fix_spans(response_entities, prompt_reports["text"][i])
    fixed_responses.append({"id": response_object["id"], "entities": fixed_entities})
# print(json.dumps(fixed_responses, indent=4, ensure_ascii=False))

# guardar resultados en un archivo jsonl en la carpeta actual, append mode
output_file_path = "output.jsonl"

with open(output_file_path, "a") as f:
    for item in fixed_responses:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

### Evaluación de desempeño

##### Funciones

In [ ]:
def compare_spans_iou(span1, span2):
    start1, end1 = span1
    start2, end2 = span2
    intersection = max(0, min(end1, end2) - max(start1, start2))
    union = min(max(end1, end2) - min(start1, start2), end1-start1 + end2-start2)
    return intersection/union

def compare_entities(entity1, entity2):
    start_offset_distance_threshold = 20
    iou_threshold = 0.5
    entities_are_equal = False
    # si las entidades no tienen el mismo label, no son iguales
    if entity1["label"] != entity2["label"]:
        return entities_are_equal
    # si el iou entre los spans de las entidades es menor al umbral definido, no son iguales
    if compare_spans_iou([entity1["start_offset"], entity1["end_offset"]], [entity2["start_offset"], entity2["end_offset"]]) < iou_threshold:
        return entities_are_equal
    else:
        # si el iou es mayor al umbral, chequear si el comienzo de ambas está a menos de una distancia umbral
        if abs(entity1["start_offset"] - entity2["start_offset"]) > start_offset_distance_threshold:
            return entities_are_equal

    entities_are_equal = True
    return entities_are_equal

# contar tp, fp, fn para cada clase para un reporte
def calculate_tp_fp_fn_report(gold_entities, generated_entities, classes):
    num_classes = len(classes)
    tp = [0]*num_classes
    fp = [0]*num_classes
    fn = [0]*num_classes

    for gold_entity in gold_entities:
        found = False
        for generated_entity in generated_entities:
            if compare_entities(gold_entity, generated_entity):
                tp[classes.index(gold_entity["label"])] += 1
                found = True
                break
        if not found:
            fn[classes.index(gold_entity["label"])] += 1

    for generated_entity in generated_entities:
        found = False
        for gold_entity in gold_entities:
            if compare_entities(gold_entity, generated_entity):
                found = True
                break
        if not found:
            if generated_entity["label"] in classes:
                fp[classes.index(generated_entity["label"])] += 1

    return tp, fp, fn

# calcular precision, recall y f1 dados tp, fp y fn
def calculate_metrics(tp, fp, fn):
    precision = 0.0
    recall = 0.0
    f1 = 0.0

    precision = tp/(tp+fp) if tp+fp > 0 else 0
    recall = tp/(tp+fn) if tp+fn > 0 else 0
    f1 = 2*precision*recall/(precision+recall) if precision+recall > 0 else 0

    return precision, recall, f1

##### Ejecución (cálculo de métricas)

In [ ]:
# leer entidades corregidas de los informes desde el archivo jsonl output.jsonl
fixed_responses = []
with open(output_file_path, "r") as f:
    for line in f:
        fixed_responses.append(json.loads(line))

print("Número de informes con entidades corregidas: ", len(fixed_responses))

In [ ]:
# create or open csv file to save results
results_file_path = data_path + "NER/results.csv"
if not os.path.exists(results_file_path):
    results_df = pd.DataFrame(columns=["chat_model","fecha/hora", "num_reports", "macro_f1", "micro_f1", "HALL_presente_f1", "HALL_ausente_f1", "CARACT_f1", "CUAD_f1", "LAT_f1", "REG_f1", "DENS_f1"])
    results_df.to_csv(results_file_path, index=False)
else:
    results_df = pd.read_csv(results_file_path)

# calcular tp, fp, fn para cada clase
classes = ["HALL_presente", "HALL_ausente", "CARACT", "CUAD", "LAT", "REG", "DENS"]
gold_entities_counts = {}
tp_total = [0]*len(classes)
fp_total = [0]*len(classes)
fn_total = [0]*len(classes)

print("Scores por clase")
        
# por cada informe, llamar a la función calculate_tp_fp_fn_report. Sumar los resultados de cada informe para calcular tp, fp, fn totales.
for i, informe in enumerate(prompt_reports["text"][:NUM_REPORTS]):
    gold_entities = prompt_reports["entities"][i]
    # eliminar entidades de tipo "GANGLIOS"
    gold_entities = [entity for entity in gold_entities if entity["label"] != "GANGLIOS"]
    # contar cantidad de entidades de cada clase
    for entity in gold_entities:
        if entity["label"] in gold_entities_counts:
            gold_entities_counts[entity["label"]] += 1
        else:
            gold_entities_counts[entity["label"]] = 1

    # buscar entidades corregidas del informe, buscando por id en la lista fixed_responses
    generated_entities = []
    for response in fixed_responses:
        if response["id"] == prompt_reports["id"][i]:
            generated_entities = response["entities"]
    if len(generated_entities) == 0:
        print("No se encontraron entidades corregidas para el informe ", i)
        continue
    tp, fp, fn = calculate_tp_fp_fn_report(gold_entities, generated_entities, classes)
    tp_total = [sum(x) for x in zip(tp_total, tp)]
    fp_total = [sum(x) for x in zip(fp_total, fp)]
    fn_total = [sum(x) for x in zip(fn_total, fn)]

# calcular precision, recall y f1 para cada clase
metrics_per_class = []
for i, class_name in enumerate(classes):
    print("Clase: ", class_name)
    metrics = calculate_metrics(tp_total[i], fp_total[i], fn_total[i])
    print("Precision: ", metrics[0])
    print("Recall: ", metrics[1])
    print("F1: ", metrics[2])
    metrics_per_class.append(metrics)
    # mostrar cantidad de entidades de la clase
    if class_name in gold_entities_counts:
        print("Cantidad de entidades de la clase: ", gold_entities_counts[class_name])

# macro-average
macro_precision = 0
macro_recall = 0
macro_f1 = 0

for i, class_name in enumerate(classes):
    macro_precision += metrics_per_class[i][0]/len(classes)
    macro_recall += metrics_per_class[i][1]/len(classes)
    macro_f1 += metrics_per_class[i][2]/len(classes)

print("\nPromedios")


print("Macro-average")
print("Precision: ", macro_precision)
print("Recall: ", macro_recall)
print("F1: ", macro_f1)

# micro-average

micro_precision, micro_recall, micro_f1 = calculate_metrics(sum(tp_total), sum(fp_total), sum(fn_total))

print("Micro-average")
print("Precision: ", micro_precision)
print("Recall: ", micro_recall)
print("F1: ", micro_f1)

In [ ]:
# guardar resultados en csv
import datetime
results_df = pd.concat([results_df, pd.DataFrame([[CHAT_MODEL,datetime.datetime.now(), NUM_REPORTS, macro_f1, micro_f1, metrics_per_class[0][2], metrics_per_class[1][2], metrics_per_class[2][2], metrics_per_class[3][2], metrics_per_class[4][2], metrics_per_class[5][2], metrics_per_class[6][2]]], columns=results_df.columns)], ignore_index=True)


results_df.to_csv(results_file_path, index=False)